# preprocessing


## imports

In [2]:
import pandas as pd
import numpy as np

import re
import random
from  itertools import chain

import joblib

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw



from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier, GradientBoostingClassifier

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

## creating csv


In [13]:
dict_2 = {
          'Ставка': [ 'ставк', 'процент', 'доля'],

          'ПВ': [ 'первый', 'взнос', 'начальный', 'первоначальный', 'платеж', 'выплата', 'плата', 'стартовый']
}




dict_4 = {
        'Новостройки': ['для нового жилья', 'кредит на новую квартиру' ,'ипотечный кредит новостройка',
                      'ипотека новый дом', 'ипотека строящийся дом', 'ипотека новостройка',
                      'ипотека новостройка ' ,'ипотека на новую квартиру',],
        
        'Вторичное жильё': ['ипотека на вторичное жилье в москве','ипотека для квартиры в старом доме', 'ипотека старая квартира',
                          'кредит старая квартира', 'ипотека бу дом', 'ипотека б/у дом', 'ипотека бу дом', 'ипотека бу квартира',
                          'ипотека бу квартира', 	'ипотека на бывшую в употреблении квартиру',
                          'ипотека для бывший в употреблении дом ' ], 
        
        'Семейная ипотека': ['ипотечный кредит для семьи', 'семейный ипотечный кредит',
                          'ипотека для большой семьи','по ипотеке для семьи',' семейная ипотека',# 
                          'ипотека для семей с детьми', 'ипотека для молодой семьи',],
      
        
        'Ипотека с господдержкой': ['ипотека с господдержкой', 'льготная ипотека','ипотека государственная поддержка',
                                ' по ипотеке с государственной поддержкой','ипотека с государственной поддержкой',
                                'ипотечный кредит с государственной поддержкой', 'кредит на жилье с государственной поддержкой', 
                                'кредит на жилье льготный', 'кредит на жилье с господдержкой ', ]
}

def key_by_val(x, dict_ = dict_4): # поиск ключа по значению
  for k, v in dict_.items():
    if x in v:
        return k

df = pd.DataFrame(list(chain.from_iterable(dict_4.values())), columns = ['quests'])
df['cats'] = df['quests'].apply(key_by_val)

list_of_cats2 = list(chain.from_iterable(dict_2.values()))
temp_df = df.copy()

for i in list_of_cats2:
  s = temp_df['quests'] + ' ' + i
  df = df.append(pd.concat([s, temp_df['cats']], axis=1))
  # добавление ключевых слов к каждой строке чтобы они меньше влияли на модель

df.reset_index(inplace=True, drop=True)

In [14]:
df.shape

(420, 2)

## augmentation


Двойной перевод был убран из-за слишком долгого времени выполнения, изначально была еще символьная аугментация, она лишь расширяла словарь и ухудшала результат


In [16]:
context_emb_aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', top_k=15, aug_p=0.05, stopwords=['б/у', 'бу'], aug_max = 2, batch_size=16)

# dt_aug = naw.back_translation.BackTranslationAug(from_model_name='Helsinki-NLP/opus-mt-ru-fr', to_model_name='Helsinki-NLP/opus-mt-fr-ru', batch_size=16)

emb_aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-uncased', top_k=15, aug_p=0.05, stopwords=['б/у', 'бу'], aug_max = 2, batch_size=16)


aug_list_text = [context_emb_aug, emb_aug, emb_aug]




for i in aug_list_text:
  df_ = df.copy()
  df_['quests'] = df.quests.apply(lambda x: i.augment(x)[0])
  df = df.append(df_, ignore_index=True).drop_duplicates(['quests'])


In [21]:
df.sample(10)

,quests,cats
1993,лес или жилье льготныи доля,Ипотека с господдержкой
814,кредит на языке льготныи,Ипотека с господдержкой
1325,дом бу кв первоначальныи,Вторичное жильё
729,ипотека на новую эпоху плата,Новостройки
1212,кеп бу кв доля,Вторичное жильё
1619,ипотека с петр первыи,Ипотека с господдержкой
780,по ипотеке с научнои поддержкои стартовыи,Ипотека с господдержкой
2545,ипотека все квартиры во старом доме стартовыи,Вторичное жильё
1408,за плата,Новостройки
2681,доля доля,Семейная ипотека


In [17]:
df.shape

(2758, 2)

In [18]:
df.to_csv('data/preprocessed_full_no_char_aug.csv')


## encoding

In [13]:
df = pd.read_csv('data/preprocessed_full_no_char_aug.csv',  index_col=[0])


encoder = LabelEncoder()
y = encoder.fit_transform(df['cats'])

df['quests'] = df.quests.apply(lambda x: re.sub(r'([^А-Яа-я\s]+)','', x))
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['quests'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

# training

### models

In [14]:
i = 'weighted'
a = 'ovr'
model_1 = SVC(probability=True)
model_2 = LogisticRegression(max_iter=500, solver='saga')
model_3 =  SGDClassifier(loss = 'modified_huber')


model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)


y_pred_1 = model_1.predict(X_test)
y_score_1 = model_1.predict_proba(X_test)

y_pred_2 = model_2.predict(X_test)
y_score_2 = model_2.predict_proba(X_test)

y_pred_3 = model_3.predict(X_test)
y_score_3 = model_3.predict_proba(X_test)


print(f'SVC accuracy: {round(accuracy_score(y_test, y_pred_1), 3)}, f1: {round(f1_score(y_test, y_pred_1, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score_1, multi_class = a), 3)}')

print(f'LR accuracy: {round(accuracy_score(y_test, y_pred_2), 3)}, f1: {round(f1_score(y_test, y_pred_2, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score_2, multi_class = a), 3)}')

print(f'SGDC accuracy: {round(accuracy_score(y_test, y_pred_3), 3)}, f1: {round(f1_score(y_test, y_pred_3, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score_3, multi_class = a), 3)}')

SVC accuracy: 0.935, f1: 0.935, roc auc:0.99
LR accuracy: 0.926, f1: 0.927, roc auc:0.992
SGDC accuracy: 0.917, f1: 0.918, roc auc:0.987


## gridsearch 


In [15]:
#значения выбраны после нескольких тестов близкие к оптимальным значениям
params_1 = {
    'gamma': ['scale', 'auto'],
    'C' : [.5,.7, 1, 2, 3],
    }

params_3 = {
    'max_iter': [1000, 1500, 2000, 2500, 3000, 3500]
}

cv_1 = GridSearchCV(model_1, param_grid=params_1, scoring= 'f1_macro')
cv_3 = GridSearchCV(model_3, param_grid=params_3, scoring='f1_macro')

cv_1.fit(X_train, y_train)
cv_3.fit(X_train, y_train)

GridSearchCV(estimator=SGDClassifier(loss='modified_huber'),
             param_grid={'max_iter': [1000, 1500, 2000, 2500, 3000, 3500]},
             scoring='f1_macro')

In [16]:
model_1 = SVC(probability=True, **cv_1.best_params_)
model_3 = SGDClassifier(loss = 'modified_huber', **cv_3.best_params_)

model_1.fit(X_train, y_train)
model_3.fit(X_train, y_train)

y_pred_1 = model_1.predict(X_test)
y_score_1 = model_1.predict_proba(X_test)

y_pred_3 = model_3.predict(X_test)
y_score_3 = model_3.predict_proba(X_test)

print(f'tunned SVC accuracy: {round(accuracy_score(y_test, y_pred_1), 3)}, f1: {round(f1_score(y_test, y_pred_1, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score_1, multi_class = a), 3)}')

print(f'tunned SGDC accuracy: {round(accuracy_score(y_test, y_pred_3), 3)}, f1: {round(f1_score(y_test, y_pred_3, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score_3, multi_class = a), 3)}')

tunned SVC accuracy: 0.935, f1: 0.935, roc auc:0.99
tunned SGDC accuracy: 0.92, f1: 0.92, roc auc:0.986


## ensembles

In [17]:
estimators = [('svc' , model_1),
              ('lr', model_2),
              ('sgd', model_3),
              ]
              
model = VotingClassifier(estimators=estimators, voting='soft')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_score = model.predict_proba(X_test)

print(f'Voting accuracy: {round(accuracy_score(y_test, y_pred), 3)}, f1: {round(f1_score(y_test, y_pred, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score, multi_class = a), 3)}')

Voting accuracy: 0.93, f1: 0.931, roc auc:0.993


In [18]:
model = StackingClassifier(estimators=estimators)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_score = model.predict_proba(X_test)

print(f'Stacking accuracy: {round(accuracy_score(y_test, y_pred), 3)}, f1: {round(f1_score(y_test, y_pred, average=i), 3)}, \
roc auc:{round(roc_auc_score(y_test, y_score, multi_class = a), 3)}')

Stacking accuracy: 0.935, f1: 0.935, roc auc:0.993


Наилучшие результаты у svc и stacking моделей, буду использовать первую из-за ее простоты.
Какже были опробованы decision tree и random forest, Gradienboosting, но они показали себя значительно хуже линейных моделей, из финальной версии были убраны




## relearning with full df and model saving 



In [20]:
model_1.fit(X, y)


joblib.dump(model_1, 'models/model_svc.sav')
joblib.dump(encoder, 'models/encoder.sav')
joblib.dump(vectorizer, 'models/vectorizer.sav')


['models/vectorizer.sav']